In [10]:
# %pip install pymilvus

In [11]:
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

# Connect to the Milvus server
connections.connect("default", host="localhost", port="19530")

# Check if connected
if not connections.has_connection("default"):
    raise Exception("Failed to connect to Milvus")

# Define a simple schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=128)
]
schema = CollectionSchema(fields, "example collection")

# Create a collection
collection = Collection(name="example_collection", schema=schema)

# Insert data into the collection
import numpy as np

data = [
    [np.random.random(128).tolist() for _ in range(10)]  # embeddings
]
collection.insert(data)

# Flush to ensure data is persisted
collection.flush()

# Create an index on the collection
index_params = {
    "metric_type": "L2",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 1024}
}
collection.create_index(field_name="embedding", index_params=index_params)

# Load the collection into memory
collection.load()

# Perform a search
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10}
}
vectors_to_search = [np.random.random(128).tolist()]
results = collection.search(vectors_to_search, "embedding", param=search_params, limit=5)

# Display search results
for result in results:
    print(f"Distance: {result.distances}, ID: {result.ids}")

# Disconnect
connections.disconnect("default")


Distance: [18.10807228088379, 18.551435470581055, 19.042877197265625, 19.378337860107422, 19.38062286376953], ID: [450012791209465163, 450023491085533378, 450023491085533385, 450012791209465143, 450023491085533384]
